In [1]:
%load_ext autoreload
%autoreload 1

from google.colab import drive
drive.mount("/content/drive")
# Direct to your assignment folder.
%cd /content/drive/MyDrive/Cartonomics/Cocass

Mounted at /content/drive
/content/drive/MyDrive/Cartonomics/Cocass


In [2]:
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from PIL import Image
import random
import os

# Path to your COCO dataset
annotation_file = "data/coco_datasets/train_data/train_annotations.json"
image_dir = "data/coco_datasets/train_data/train_images"

# Number of images to display per figure
grid_size = 5  # 5x5 grid
num_images = grid_size ** 2  # Total images in one grid

# Load COCO annotations
coco = COCO(annotation_file)
image_ids = coco.getImgIds()
selected_images = random.sample(image_ids, num_images)

# --- Function to create the figure with full-size images ---
def create_full_size_figure(image_ids, grid_size, title, save_path):
    fig, axs = plt.subplots(grid_size, grid_size, figsize=(15, 15))
    axs = axs.flatten()

    for i, img_id in enumerate(image_ids):
        img_info = coco.loadImgs(img_id)[0]
        img_path = os.path.join(image_dir, img_info['file_name'])
        img = Image.open(img_path)
        axs[i].imshow(img)
        axs[i].axis("off")
        axs[i].set_title(f"Image ID: {img_id}", fontsize=8)

    for ax in axs[len(image_ids):]:  # Hide any unused subplots
        ax.axis("off")

    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

# --- Function to create the figure with cropped regions ---
def create_cropped_figure(image_ids, grid_size, title, save_path):
    fig, axs = plt.subplots(grid_size, grid_size, figsize=(15, 15))
    axs = axs.flatten()

    for i, img_id in enumerate(image_ids):
        img_info = coco.loadImgs(img_id)[0]
        img_path = os.path.join(image_dir, img_info['file_name'])
        img = Image.open(img_path)

        # Load annotations for the image
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)

        if anns:
            # Use the bounding box of the first annotation as the crop region
            x, y, w, h = anns[0]['bbox']
            cropped_img = img.crop((x, y, x + w, y + h))
        else:
            # Fallback to a random crop if no annotations exist
            width, height = img.size
            cropped_img = img.crop((width // 4, height // 4, 3 * width // 4, 3 * height // 4))

        axs[i].imshow(cropped_img)
        axs[i].axis("off")
        axs[i].set_title(f"Image ID: {img_id}", fontsize=8)

    for ax in axs[len(image_ids):]:  # Hide any unused subplots
        ax.axis("off")

    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

# Generate and save the figures
create_full_size_figure(selected_images, grid_size, "Full-Size Images from Dataset", "full_size_figure.png")
create_cropped_figure(selected_images, grid_size, "Cropped Regions from Dataset", "cropped_regions_figure.png")


loading annotations into memory...
Done (t=0.78s)
creating index...
index created!


In [8]:
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from PIL import Image
import random
import os

# Path to your COCO dataset
annotation_file = "data/coco_datasets/train_data/train_annotations.json"
image_dir = "data/coco_datasets/train_data/train_images"

# Number of images to display per figure
grid_size = 5  # 5x5 grid
num_images = grid_size ** 2  # Total images in one grid

# Load COCO annotations
coco = COCO(annotation_file)
image_ids = coco.getImgIds()

# Filter to ensure images are not corrupt
def filter_valid_images(image_ids):
    valid_images = []
    for img_id in image_ids:
        img_info = coco.loadImgs(img_id)[0]
        img_path = os.path.join(image_dir, img_info['file_name'])
        try:
            img = Image.open(img_path)
            img.verify()  # Verify if the image is valid
            valid_images.append(img_id)
        except Exception:
            continue
    return valid_images

valid_image_ids = filter_valid_images(image_ids)

# --- Function to create the figure with full-size images ---
def create_full_size_figure(image_ids, grid_size, title, save_path):
    fig, axs = plt.subplots(grid_size, grid_size, figsize=(15, 15), gridspec_kw={'wspace': 0.02, 'hspace': 0.02})
    axs = axs.flatten()

    for i, img_id in enumerate(image_ids):
        img_info = coco.loadImgs(img_id)[0]
        img_path = os.path.join(image_dir, img_info['file_name'])
        img = Image.open(img_path)
        axs[i].imshow(img)
        axs[i].axis("off")

    for ax in axs[len(image_ids):]:  # Hide any unused subplots
        ax.axis("off")

    # Add row labels (A, B, C, ...)
    for row in range(grid_size):
        row_center = 0.92 - row * 0.18 + 0.09  # Center alignment with slight spacing
        fig.text(0.04, row_center, chr(65 + row), ha="center", va="center", fontsize=12, rotation=0)

    # Add column labels (1, 2, 3, ...)
    for col in range(grid_size):
        col_center = 0.1 + col * 0.18 - 0.09  # Center alignment with slight spacing
        fig.text(col_center, 0.94, str(col + 1), ha="center", va="center", fontsize=12, rotation=0)

    plt.suptitle(title, fontsize=16)
    plt.show()
    plt.savefig(save_path)
    plt.close()

# Generate multiple full-size figures
for i in range(3):  # Generate 3 figures to choose from
    selected_images = random.sample(valid_image_ids, num_images)
    create_full_size_figure(selected_images, grid_size, f"Full-Size Images from Dataset {i+1}", f"full_size_figure_{i+1}.png")


Output hidden; open in https://colab.research.google.com to view.